In [2]:
import sklearn, transformers, torch, tqdm, stanza, jieba
import numpy as np
import pandas as pd

/home/chwu/projects/gptdev/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('grave1.txt', 'r', encoding = "utf-16") as file:
    grave1 = file.read()
with open('grave2.txt', 'r', encoding = "utf-8") as file:
    grave2 = file.read()
with open('grave3.txt', 'r', encoding = "utf-8") as file:
    grave3 = file.read()
with open('grave4.txt', 'r', encoding = "utf-8") as file:
    grave4 = file.read()
with open('grave5.txt', 'r', encoding = "utf-8") as file:
    grave5 = file.read()
with open('grave6.txt', 'r', encoding = "utf-8") as file:
    grave6 = file.read()
with open('grave7.txt', 'r', encoding = "utf-8") as file:
    grave7 = file.read()
with open('grave8.txt', 'r', encoding = "utf-8") as file:
    grave8 = file.read()
with open('grave9.txt', 'r', encoding = "utf-8") as file:
    grave9 = file.read()
    
text = grave1+grave2+grave3+grave4+grave5+grave6+grave7+grave8+grave9

In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

def create_bert_dict(text):
    tokens = tokenizer.tokenize(text)
    token_index_dict = {}
    current_order = 0
    for token in tokens:
        if token not in token_index_dict:
            token_index_dict[token] = current_order
            current_order += 1
    return token_index_dict

bert_dict = create_bert_dict(text)
print(list(bert_dict.items())[:10])
print(list(bert_dict.items())[-10:])
print(len(bert_dict))

[('～～～', 0), ('##～', 1), ('小', 2), ('说', 3), ('下', 4), ('载', 5), ('尽', 6), ('在', 7), ('http', 8), (':', 9)]
[('灿', 4180), ('砧', 4181), ('锏', 4182), ('嘻', 4183), ('糗', 4184), ('悼', 4185), ('愫', 4186), ('儡', 4187), ('迢', 4188), ('鹞', 4189)]
4190


In [5]:
bert_stoi = bert_dict
bert_itos = {index: char for char, index in bert_dict.items()}
encode_bert = lambda s: [bert_stoi[c] for c in s if c in bert_stoi]
decode_bert = lambda l: ''.join([bert_itos[i] for i in l if i in bert_itos])

In [6]:
import torch
bert_data = torch.tensor(encode_bert(text), dtype=torch.long)
train_data = bert_data[:int(0.9*len(bert_data))]
val_data = bert_data[int(0.9*len(bert_data)):]

In [11]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 9000
eval_interval = 100
learning_rate = 8e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 8
n_layer = 4
dropout = 0.2
vocab_size = len(bert_dict)

In [8]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [22]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)

1.885022 M parameters
step 0: train loss 8.5050, val loss 8.5011
step 100: train loss 4.9440, val loss 4.9933
step 200: train loss 4.6260, val loss 4.6834
step 300: train loss 4.4938, val loss 4.5881
step 400: train loss 4.4122, val loss 4.5237
step 500: train loss 4.3558, val loss 4.4955
step 600: train loss 4.3162, val loss 4.4387
step 700: train loss 4.2825, val loss 4.4340
step 800: train loss 4.2545, val loss 4.4032
step 900: train loss 4.2206, val loss 4.3795
step 1000: train loss 4.2096, val loss 4.3645
step 1100: train loss 4.1610, val loss 4.3464
step 1200: train loss 4.1418, val loss 4.3367
step 1300: train loss 4.1360, val loss 4.3302
step 1400: train loss 4.1183, val loss 4.3247
step 1500: train loss 4.1064, val loss 4.2940
step 1600: train loss 4.0661, val loss 4.2858
step 1700: train loss 4.0524, val loss 4.2462
step 1800: train loss 4.0466, val loss 4.2351
step 1900: train loss 4.0020, val loss 4.2160
step 2000: train loss 3.9669, val loss 4.1772
step 2100: train loss 3.

In [23]:
print(decode_bert(m.generate(context, max_new_tokens=300)[0].tolist()))

～～～，对这种程有浅浅不见过天浪的时候，也还没有办法，又一路点，就是深度得更凌云顶天宫，我们借奇怪吗？我看着我们，其推测奇心，我们就是天这样子，可以知道这里的不存心理丹药，但是再跑出了。我跑了一后，因为走的上在岸边进，我几乎能看到很多事情的确是阴谋，簧火着洞里的裂谷了状态，河达出来的很明砖顶都有这一种安处修建的洞壁走底，但是我们放松，门上更加的小命，特几个都有改毙的破坏心眼，而一路块的漩涡了，不能睡成任何的感觉。这种走形且步已经有的理子，下头被炸开，冰谷边海不知道湖边在他们挖掘，必然会围林子的宽度，开了七星期，右手电柄都跟上来。所以很大事，但是不应该不会再偷估计都给你们在想要这样，碰上能肯定有涂碍。


In [20]:
test1 = '冥殿四面牆壁倒不是什麽都没有，只有些打底的壁画，都是白描，还没有进行上色，画的是日月星辰。'

test2 = "该局高度重视，立即组织执法人员对该事件进行调查，对当事摊主进行了问询，摊主已承认其存在缺斤少两行为。"

test3 = "面自童奶里且平由干笑五浪，禾尺要见姐语绿吗民隻实蝶各、反下反第几她次视亭。"

test4 = "崩贴喘灵存匈铸同刃啬曾贝垮邓？敷仰吁左骗盘备。插护，炭泌倒，臭酿层述？"

encodings = torch.tensor(encode_bert(test1), dtype=torch.long).unsqueeze(0).to(device)
input_ids = encodings[:, 0:block_size].to(device)
target_ids = input_ids.clone()
with torch.no_grad():
  outputs, loss = m(input_ids, targets=target_ids)
outputs = outputs.unsqueeze(0)
probs = F.softmax(outputs, dim=-1)
torch.manual_seed(1337)
import torchmetrics
from torchmetrics.text import Perplexity

perp = Perplexity(ignore_index=-100).to(device)
perp(outputs, target_ids)


tensor(24963.7305)